In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
sentiment_texts_df: pd.DataFrame = pd.read_pickle("../data/sentiment_texts.pkl")
names_and_synonyms_df = pd.read_excel("../data/names_n_synonyms.xlsx")

In [4]:
names_and_synonyms_df["NAMINGS"] = names_and_synonyms_df.iloc[:, 3:].apply(
    lambda row: row.dropna().tolist(), axis=1
)
names_and_synonyms_df = names_and_synonyms_df.drop(columns=names_and_synonyms_df.columns[2:-1])
names_and_synonyms_df = names_and_synonyms_df.set_index("issuerid")
namings = names_and_synonyms_df["NAMINGS"]

In [5]:
sentiment_texts_df["MessageID_copy"] = sentiment_texts_df["MessageID"]
sentiment_texts_df["MessageID_copy"] = sentiment_texts_df["MessageID"]
sentiment_texts_df = (
    sentiment_texts_df.groupby("MessageID_copy")
    .agg(list)
    .map(lambda x: x[0] if (len(x) == 1) else x)
    .drop_duplicates(subset=["MessageID"])
    .reset_index(drop=True)
)
sentiment_texts_df[["issuerid", "SentimentScore"]] = sentiment_texts_df[
    ["issuerid", "SentimentScore"]
].map(lambda x: x if isinstance(x, list) else [x])

In [ ]:
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")

In [8]:
results = dict()
ind = 0
for _, row in tqdm(sentiment_texts_df.iterrows(), total=len(sentiment_texts_df)):
    if ind > 500:
        break
    message_id = row["MessageID"]
    message_text = row["MessageText"]
    if isinstance(message_text, list):
        message_text = message_text[0]
        message_id = message_id[0]
    labels = ["Company"]
    entities = model.predict_entities(message_text, labels=labels)
    for entity in entities:
        entity_text = entity["text"]
        found_issuer_id = namings.apply(
            lambda x: entity_text.lower() in [s.lower() for s in x]
        ).argmax()
        if found_issuer_id > 0:
            results.setdefault(message_id, set()).add(found_issuer_id + 1)
    ind += 1

  6%|▌         | 429/6889 [03:29<52:34,  2.05it/s]  


KeyboardInterrupt: 

In [34]:
real_results = dict()
for x in sentiment_texts_df[["MessageID", "issuerid"]].to_numpy():
    mid = x[0]
    if isinstance(mid, list):
        mid = mid[0]
    real_results.setdefault(mid, set()).update(x[1])

In [35]:
accuracy_score = []
iids: set
for mid, iids in real_results.items():
    acc_sc = max(len(iids.intersection(results[mid] if mid in results else {})), 1) / len(iids)
    if mid in results:
        accuracy_score.append(acc_sc)

In [36]:
np.mean(accuracy_score)

0.9133274301456119

In [37]:
len(results)

275

In [43]:
sentiment_texts_df[sentiment_texts_df.MessageID == 525]

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,525,1564218803,[26],[5],2023-06-02 19:25:01,2023-06-02 19:09:06,Результаты фондов под управлением УК Арикапита...,False


In [38]:
results

{525: {4, 7, 26, 48, 56, 111, 112, 163},
 559: {129},
 561: {99},
 562: {90, 99, 116, 152},
 567: {129},
 581: {90, 99, 116, 152},
 609: {111, 160},
 621: {7, 111},
 622: {90, 99, 116, 152},
 623: {7},
 641: {111, 160},
 648: {111, 115, 163},
 654: {236},
 659: {90, 99, 116, 152},
 672: {90, 99, 116, 152},
 674: {111, 160},
 679: {90, 99, 116, 152},
 733: {116},
 772: {91, 100, 142},
 779: {7, 142, 241},
 1236: {235},
 1275: {4,
  9,
  11,
  22,
  24,
  48,
  62,
  67,
  79,
  86,
  99,
  111,
  112,
  113,
  114,
  115,
  116,
  127,
  129,
  152,
  153,
  163,
  185,
  187,
  197,
  199,
  225,
  227,
  237},
 1287: {115},
 1344: {116},
 1379: {235},
 1380: {112},
 1381: {112},
 1394: {152},
 1395: {152},
 1411: {32},
 1423: {48},
 1431: {53, 56, 241},
 1436: {7},
 1437: {7},
 1440: {4, 58, 100, 111, 116, 127, 129, 150, 166, 199},
 1442: {4, 11, 24, 53, 90, 100, 116, 152, 187, 236},
 1451: {48},
 1454: {61},
 1457: {236},
 1482: {61},
 1485: {4,
  24,
  36,
  53,
  86,
  90,
  103,
 